In [24]:
import numpy as np
import open3d as o3d

def display_inlier_outlier(cloud, ind):
    inlier_cloud = cloud.select_by_index(ind)
    outlier_cloud = cloud.select_by_index(ind, invert=True)

    print("Showing outliers (red) and inliers (gray): ")
    # outlier_cloud.paint_uniform_color([1, 0, 0])
    inlier_cloud.paint_uniform_color([0.8, 0.8, 0.8])
    # o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud],
    #                                   zoom=0.3412,
    #                                   front=[0.4257, -0.2125, -0.8795],
    #                                   lookat=[2.6172, 2.0475, 1.532],
    #                                   up=[-0.0694, -0.9768, 0.2024])
    o3d.visualization.draw_geometries([inlier_cloud],
                                      zoom=0.3412,
                                      front=[0.4257, -0.2125, -0.8795],
                                      lookat=[2.6172, 2.0475, 1.532],
                                      up=[-0.0694, -0.9768, 0.2024])
    
    return inlier_cloud



old_name = f"dense.0.ply"
pcd = o3d.io.read_point_cloud(old_name)
# og_cloud.colors = o3d.utility.Vector3dVector(rgb_t.astype(np.float) / 255.0)

In [25]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [2]:
# test = pcd.uniform_down_sample(1000)
# test = pcd.uniform_down_sample(150)
# test = pcd.uniform_down_sample(75)
sz = .02
# sz = .2
test=pcd.voxel_down_sample(sz)
# o3d.visualization.draw([test])
# o3d.visualization.draw_geometries([test])

In [3]:
# sz = .02
n_neigh = 5
std_ratio = 0.25


n_pts = 12
rad = .15


_, sind = test.remove_statistical_outlier(n_neigh, std_ratio, False)
_, rind = test.remove_radius_outlier(n_pts, rad)


In [4]:
inl = display_inlier_outlier(test, sind)
# display_inlier_outlier(test, rind)

Showing outliers (red) and inliers (gray): 
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [5]:
print(inl)

PointCloud with 146083 points.


In [5]:
_, rind = inl.remove_radius_outlier(n_pts, rad)

In [6]:
display_inlier_outlier(inl, rind)
# o3d.visualization.draw([test])

Showing outliers (red) and inliers (gray): 
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


PointCloud with 144861 points.

: 

In [19]:
dists = inl.compute_nearest_neighbor_distance()
avg_dist = np.mean(dists)
# bpa_rad = 3*avg_dist
bpa_rad = avg_dist


bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    inl, o3d.utility.DoubleVector([bpa_rad, bpa_rad * 2])
)


# dec_mesh = bpa_mesh.simplify_quadratic_decimation(10000000)

In [12]:
p_mesh,_ = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(inl, depth=9)

In [22]:
# alpha = .03
alpha = .5
a_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(inl, alpha)



In [23]:
a_mesh.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([a_mesh], window_name="alph mesh")

bpa_mesh.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([bpa_mesh], window_name="bpa mesh")

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [14]:
p_mesh.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([p_mesh], window_name="pois mesh")

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
# bpa_rad = 3 * avg_dist
# test = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
#     inl, o3d.utility.DoubleVector([bpa_rad, bpa_rad * 2])
# )

TypeError: create_from_point_cloud_ball_pivoting(): incompatible function arguments. The following argument types are supported:
    1. (pcd: open3d.cpu.pybind.geometry.PointCloud, radii: open3d.cpu.pybind.utility.DoubleVector) -> open3d.cpu.pybind.geometry.TriangleMesh

Invoked with: TriangleMesh with 146083 points and 150802 triangles., DoubleVector[0.0275777, 0.0551553]

In [7]:
type(p_mesh[0])

open3d.cpu.pybind.geometry.TriangleMesh

In [13]:
temp = o3d.t.geometry.TriangleMesh.from_legacy(bpa_mesh)

In [30]:
filled = temp.fill_holes()

[Open3D WARNING] Ignoring attribute 'normals' for TensorMap with primary key 'indices'


In [39]:
# leg = o3d.geometry.TriangleMesh()
# # leg.vertices = filled.vertices.cpu()
# # leg.triangles = filled.triangles.cpu()
# leg.vertices = filled.vertex.cpu()
# leg.triangles = filled.triangle

# vis = o3d.visualization.Visualizer()
# vis.create_window()
# vis.add_geometry(temp)
# vis.run()

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


TypeError: add_geometry(): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.visualization.Visualizer, geometry: open3d.cpu.pybind.geometry.Geometry, reset_bounding_box: bool = True) -> bool

Invoked with: Visualizer with name Open3D, TriangleMesh on CPU:0 [146083 vertices (Float32) and 150802 triangles (Int64)].
Vertex Attributes: colors (dtype = Float32, shape = {146083, 3}), normals (dtype = Float32, shape = {146083, 3}).
Triangle Attributes: normals (dtype = Float32, shape = {150802, 3}).

In [38]:
filled.vertex

TensorMap(primary_key="positions") with 3 attributes:
  - colors   : shape={146083, 3}, dtype=Float32, device=CPU:0
  - normals  : shape={146083, 3}, dtype=Float32, device=CPU:0
  - positions: shape={146083, 3}, dtype=Float32, device=CPU:0 (primary)
  (Use . to access attributes, e.g., tensor_map.positions)

In [33]:
vis.destroy_window()

In [18]:
bpa_mesh.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([bpa_mesh], window_name="bpa mesh")

# ds_pcd = pcd.voxel_down_sample(.02)

NameError: name 'bpa_mesh' is not defined

In [29]:
temp = bpa_mesh

In [27]:
o3d.visualization.draw_geometries([temp], window_name="bpa mesh")

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
seg = ds_pcd.segment_plane(0.01, 3, 1000)

In [ ]:
lrg_seg= seg[0]

display_inlier_outlier(ds_pcd, seg[1])